Заполните данные о необходимых переменных:

In [9]:
Today = '29/03/24'

print_to_excel = True
print_BD_to_excel = False
print_pre_data_on_sheets = False

In [10]:
from datetime import date
import pandas as pd
import numpy as np

In [11]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

Открываю следующие файлы:

In [12]:
query = f"""select * from "RISKACCESS"."bankAccountsBalanceDaily" where "reportDate" = TO_DATE('{Today}', 'DD/MM/YY')"""
data_export = export_from_RISKCUSTOM(query)
data_export.head(2)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:80: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


,##batch_id,##deleted,##inserted,##origin_name,reportDate,holding,buCode,buCountryGroup,buCountryCode,buName,bankName,bankId,bankCountryCode,bankCountryGroup,accountCurrency,accountNumber,accountStatus,balance,balanceUsd,statementDate
0,35225071@WHPROD,N,2024-04-04 11:50:16,WHFLEX,2024-03-29,EUROCHEM,USBFC,(4) US,US,EuroChem North America Corp.,Vast,103902717,US,(4) US,USD,1087053,active,56971.92,56971.92,2024-03-15
1,35225071@WHPROD,N,2024-04-04 11:50:16,WHFLEX,2024-03-29,EUROCHEM,CHSAG,(1) CH,CH,SUEK AG (Swiss),An Binh,ABBKVN,VN,(9) other,USD,1221027172080,active,72.92,72.92,2023-08-28


Произвожу расчеты:

In [13]:
for i in ['EUROCHEM', 'SUEK']:
    data_work = data_export[data_export['holding'] == i].reset_index(drop=True)
    data_work['Segment'] = merge_SalesUnits(data_work, col='buCode', merge_col='ocpSegment')
    data_work['Period'] = '<1M'
    data_work['impairment'] = 0
    data_work['Balance_USD'] = data_work['balanceUsd']
    data_work['balanceUSD'] = data_work['Balance_USD'] * (1-data_work['impairment'])
    if i == 'SUEK':
        data_work_SUEK = data_work
    if i == 'EUROCHEM':
        data_work_Ech = data_work

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:80: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:80: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


Записываю данные в output file:

In [14]:
Output_file_SUEK = "_".join([str(date.today()), 'SUEK_Cash.xlsx'])
Output_file_Ech = "_".join([str(date.today()), 'Ech_Cash.xlsx'])
Sheet_in_output_file = 'Cash'

if print_to_excel == True:
    data_work_SUEK.to_excel(Output_file_SUEK, sheet_name = Sheet_in_output_file, index = False)
    data_work_Ech.to_excel(Output_file_Ech, sheet_name = Sheet_in_output_file, index = False)

Тестовая БД:

In [15]:
# query = f"""
# select * from "unifiedOcp"
# where "reportDate" = TO_DATE('{Today}', 'DD/MM/YY')
# and "sourceSystem" = 'BANK_ACCOUNTS_BALANCE_XLS'
# """
# data_test_bd = export_from_WHWEEK(query)

query = f"""select * from "RISKACCESS"."unifiedOcp" 
where "reportDate" = TO_DATE('{Today}', 'DD/MM/YY')
and "sourceSystem" = 'BANK_ACCOUNTS_BALANCE_XLS'
"""
data_test_bd = export_from_RISKCUSTOM(query)

data_work_test_bd = data_test_bd[['holding', 'buCode', 'effectiveCurrency', 'volumeEffectiveCurrency', '##origin_name']]

data_work_test_bd_group = data_work_test_bd.groupby(['##origin_name', 'holding',	'buCode',	'effectiveCurrency'], as_index=False).agg({'volumeEffectiveCurrency':'sum'}).sort_values(['##origin_name', 'holding',	'buCode',	'effectiveCurrency', 'volumeEffectiveCurrency'])
data_work_test_bd_group = data_work_test_bd_group[data_work_test_bd_group.volumeEffectiveCurrency != 0].reset_index(drop=True)
data_work_test_bd_group = concat_columns(data_work_test_bd_group, columns=['holding',	'buCode',	'effectiveCurrency'	])

data_first_BD = data_export[['holding', 'buCode', 'accountCurrency', 'balance']]
data_first_BD['sourse'] = 'RISKACCESS'
data_first_BD_group = data_first_BD.groupby(['sourse', 'holding',	'buCode',	'accountCurrency'], as_index=False).agg({'balance':'sum'}).sort_values(['sourse', 'holding',	'buCode',	'accountCurrency', 'balance'])
data_first_BD_group = concat_columns(data_first_BD_group, columns=['holding',	'buCode',	'accountCurrency'	])
data_first_BD_group = data_first_BD_group[data_first_BD_group.balance != 0].reset_index(drop=True)

merge_df = data_first_BD_group.merge(data_work_test_bd_group, how='outer', left_on='concat_columns', right_on='concat_columns')
merge_df[['balance', 'volumeEffectiveCurrency']] = merge_df[['balance', 'volumeEffectiveCurrency']].fillna(0).astype('int64')
merge_df['equal_balance'] = merge_df['balance'] == merge_df['volumeEffectiveCurrency']
merge_df['concat_columns'] = ''
# merge_df[((~merge_df.balance.isna()) | (merge_df.volumeEffectiveCurrency != 0)) & ((~merge_df.volumeEffectiveCurrency.isna()) | (merge_df.balance != 0))]
merge_df


C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:80: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_13720\3172580302.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_first_BD['sourse'] = 'RISKACCESS'


,sourse,holding_x,buCode_x,accountCurrency,balance,concat_columns,##origin_name,holding_y,buCode_y,effectiveCurrency,volumeEffectiveCurrency,equal_balance
0,NaN,NaN,NaN,NaN,0,,WHFLEX,EUROCHEM,7100,CHF,395183,False
1,NaN,NaN,NaN,NaN,0,,WHFLEX,EUROCHEM,7100,CNY,74088690,False
2,NaN,NaN,NaN,NaN,0,,WHFLEX,EUROCHEM,7100,EUR,133289,False
3,NaN,NaN,NaN,NaN,0,,WHFLEX,EUROCHEM,7100,GBP,16186,False
4,NaN,NaN,NaN,NaN,0,,WHFLEX,EUROCHEM,7100,HKD,840096,False
...,...,...,...,...,...,...,...,...,...,...,...,...
293,RISKACCESS,SUEK,VH00,RUB,331894371,,WHFLEX,SUEK,VH00,RUB,335195748,False
294,RISKACCESS,SUEK,W100,RUB,133905944,,WHFLEX,SUEK,W100,RUB,139433978,False
295,RISKACCESS,SUEK,Y100,RUB,185764101,,WHFLEX,SUEK,Y100,RUB,205126422,False
296,RISKACCESS,SUEK,Y300,RUB,625121903,,WHFLEX,SUEK,Y300,RUB,625121903,True


In [16]:
Output_file_bd = "_".join([str(date.today()), 'RISKACCESS_to_WHFLEX_Cash.xlsx'])
if print_BD_to_excel == True:
    write_data = {'data': [f'Слева-направо Вы ыидите выгрузку из RISKACCESS и соответсвюущие ей строки из WHFLEX на {Today}', '',
                           'Денежные потоки просуммированы по холдингу, buCode и валюте.',
                           'Если слева занчений нет, значит они представлены только в WHFLEX, если справа нет - только в RISKACCESS.',
                           'Поле equal_balance сравнивает объем денежного потока из обоих источников. Если поле равно 1\True, то денежные потоки равны']}
    df = pd.DataFrame(write_data)
    df.to_excel(Output_file_bd, sheet_name='Iinstruction', index=False, header=False)
    new_list(merge_df, Output_file_bd, sheet_name='bd')

    if print_pre_data_on_sheets == True:
        new_list(data_export, Output_file_bd, sheet_name='bankAccountsBalanceDaily')
        new_list(data_test_bd, Output_file_bd, sheet_name='unifiedOcp')
        